# Region Estimation Using Active Learning GPs

This notebook demonstrates how CSAF can be used in a machine learning based analysis. Specifically, an active learning framework (modAL) uses CSAF as an oracle to sample the state space in a way that recovers a region as a Gaussian Process. 

**Task**: Estimate the region of initial system states where a condition holds true by sampling the state space. In this case, discover what initial conditions yield a successful ground collision avoidance maneuver for the F-16 aircraft.


**Assumptions**: 
* Naive sampling policies like uniform sampling is too inefficient for the dimensionality of the state space.
* The recovered region has a sufficiently low dimensional structure.
* The recovered region’s boundary is a Gaussian process f(x) with a known covariance function k(x, x’).

\begin{equation}
f(\mathbf x) \sim \mathcal G \mathcal P (m(\mathbf x), k(\mathbf x, \mathbf x’)), \quad \sum_{i=1}^{m}\sum_{j=i}^{m} \alpha_i \alpha_j k(\mathcal x_i, \mathcal x_j) \ge 0.
\end{equation}

In [ ]:
import sys

import numpy as np
import matplotlib.pyplot as plt

import tqdm

# CSAF Imports
import csaf.config as cconf
import csaf.system as csys
import run_parallel as rp  # for now, batch simulations is not in CSAF package

# ML Imports
from modAL.models import ActiveLearner
from sklearn.gaussian_process import GaussianProcessRegressor, GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

## CSAF System Creation

In [ ]:
# create a csaf configuration out of toml
my_conf = cconf.SystemConfig.from_toml("/csaf-system//f16_shield_config.toml")

In [ ]:
from IPython.display import Image

import pathlib

plot_fname = f"pub-sub-{my_conf.name}-plot.png"

# plot configuration pub/sub diagram as a file -- proj specicies a dot executbale and -Gdpi is a valid dot
# argument to change the image resolution
my_conf.plot_config(fname=pathlib.Path(plot_fname).resolve(), prog=["dot", "-Gdpi=400"])

# display written file to notebook
Image(plot_fname, height=600)

## Analysis Parameters

In [ ]:
## User Set Parameters

# bounds to sample
bounds = [(200, 1000), 
          (np.deg2rad(2.1215), np.deg2rad(2.1215)), 
          (0.0, 0.0), 
          ((np.pi/2)*0.5, (np.pi/2)*0.5), 
          (-np.pi, np.pi), 
          (-np.pi/4, np.pi/4 ),
         (0.0, 0.0),
         (-0.5, 0.5),
         (0.0, 0.0),
         (0.0, 0.0),
         (0.0, 0.0),
         (500, 8000),
         (9, 9)]

# length scales for RBF kernel used
rbf_scales = (100, 1, 1, 1, 30, 1, 1, 1, 1, 1, 1, 1000, 1)


# initial sampling count
n_initial_sampling = 13**2 * 2

In [ ]:
## Extracted Parameters

# name of system configuration
run_name = my_conf.name

# center of plane to plot contours
x_center = np.array([b[1]/2 + b[0]/2 for b in bounds])

# name of state variables
names = my_conf.get_msg_setting("plant", "states", "msg").fields_no_header

## Helper Functions

In [ ]:
def gen_random_state(bounds):
    """create a uniformly distributed initial condition"""
    sample = np.random.rand(len(bounds))
    ranges = np.array([b[1] - b[0] for b in bounds])
    offset = np.array([- b[0] for b in bounds])
    return sample * ranges - offset

def gpr_scalar_field(cx_train, cy_train, x_center, bounds, indices, rbf_scales = rbf_scales):
    """produce a matrix of GPR scalar field values against the plane centered at x_center and spanning the
    natural basis vector indexed at indices
    """
    x_min, x_max = bounds[indices[0]]
    y_min, y_max = bounds[indices[1]]
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 50),
                             np.linspace(y_min, y_max, 50))

    kernel = 100*RBF(rbf_scales)
    clf = GaussianProcessClassifier(kernel=kernel)
    clf.fit(cx_train, cy_train)

    score = clf.score(cx_train, cy_train)
    dec = np.zeros((len(xx.ravel()), 13))
    for idx in range(0, 13):
        dec[:, idx] = x_center[idx]
    dec[:, indices[0]] = xx.ravel()
    dec[:, indices[1]] = yy.ravel()
    if hasattr(clf, "decision_function"):
            Z = clf.decision_function(dec)
    else:
            Z = clf.predict_proba(dec)[:, 1]
    Z = Z.reshape(xx.shape)
    return xx, yy, Z

## PLOTTING CODE

def plot_sample_plane(ax, names, xx, yy, zz, x_train=None, y_train=None):
    """given the output of gpr_scalar_field, create a contour plot"""
    cm = plt.cm.RdBu
    ax.contourf(xx, yy, zz, cmap=cm, alpha=.8)
    ax.set_xlabel(f"{names[0]}")
    ax.set_ylabel(f"{names[1]}")
    
def gpr_plot_field(planes, names, bounds, x_all, y_all, x_center):
    """given an array of planes defined by indices, create a multi-axis plot of them"""
    n_plots = len(planes)
    n_rows = int(np.floor(n_plots**0.5))
    n_cols =int(np.ceil(n_plots / n_rows))
    fig, ax = plt.subplots(figsize=(n_cols * 5, n_rows * 5), ncols=n_cols, nrows=n_rows)
    for idx, pl in enumerate(planes):
        ridx = idx % n_cols
        cidx = idx // n_cols
        plot_sample_plane(ax[ridx][cidx], (names[pl[0]], names[pl[1]]),  
                          *gpr_scalar_field(x_all, y_all, x_center, bounds, indices=pl))
    plt.tight_layout()    
    return fig, ax

## Create the Active Learner Oracle

In [ ]:
def ground_collision_condition(cname, outs):
        """ground collision premature termnation condition"""
        return cname == "plant" and outs["states"][11] <= 0.0
    

def validate_samples(x_samp, config, tspan=(0, 20.0)):
    """given an array of plant initial states, 
    run batches of simulations to see if they meet the termination criteria"""
    # assign samples to the system plant
    init_states = [{"plant": xi} for xi in x_samp]
    n = len(init_states)
    # send jobs to the CSAF simulation workgroup
    res = rp.run_workgroup(n, config, init_states, tspan,
                            terminating_conditions=ground_collision_condition)
    passed = [x for x,_, _ in res]
    x0 = [x['plant'] for _,_,x in res]
    return tuple([passed, x0])


## Uniform Sampling

In [ ]:
x_init = [gen_random_state(bounds) for _ in range(n_initial_sampling)]
y_train, x_train = validate_samples(x_init, my_conf)
y_train, x_train = np.array(y_train), np.array(x_train)

In [ ]:
fig, ax = gpr_plot_field(((0, 11), (0, 4), (4, 11), (4, 7)), names, bounds, x_train, y_train, x_center)
ax[0][0].set_title(f"F-16 ({run_name}) ROA Slices (initial sampling)")
plt.tight_layout()
plt.show()

## Active Learner Sampling

In [ ]:
def GP_regression_std(regressor, X):
    """GPR batch sampling policy (modified for parallel simulations)"""
    _, std = regressor.predict(X, return_std=True)
    query_idxs = std.argsort()[-16:][::-1]
    return query_idxs, X[query_idxs]

# kernel to use for active GP classification
kernel = RBF(rbf_scales)

# active learner
learner = ActiveLearner(
    estimator=GaussianProcessRegressor(kernel=kernel),
    query_strategy=GP_regression_std,
    X_training=x_train, y_training=y_train
)

# define query pool of samples
n_pool_samples = 1000
x_pool = np.array([gen_random_state(bounds) for _ in range(n_pool_samples)])

In [ ]:
xs_new = np.copy(x_train)
ys_new = np.copy(y_train)

for i in tqdm.tqdm(range(10)):
    # query for labels
    query_idxs, x_samp = learner.query(x_pool)
    x_pool = np.delete(x_pool, tuple(query_idxs), 0)
        
    # process them in the oracle
    y_train_i, x_train_i = validate_samples(x_samp, my_conf)
    y_train_i, x_train_i = np.array(y_train_i), np.array(x_train_i)
    xs_new = np.vstack((xs_new, x_train_i))
    ys_new = np.hstack((ys_new, y_train_i))
    
    # teach the active learner
    for xi, yi in zip(x_train_i, y_train_i):
        learner.teach(xi[np.newaxis, :], [yi])

In [ ]:
fig, ax = gpr_plot_field([(0, 11), (0, 4), (4, 11), (4, 7)], names, bounds, xs_new[:240], ys_new[:240], x_center)
ax[0][0].set_title(f"F-16 ({run_name}) ROA Slices")
plt.tight_layout()
plt.show()